In [ ]:
from dataclasses import dataclass

import torch
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-1.5-8b-instruct-2505")
tokenizer.padding_side = 'right'
tokenizer.truncation_side='right'
tokenizer.add_eos_token = True

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(
    "kakaocorp/kanana-1.5-8b-instruct-2505",
    num_labels=2,
    #quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False

In [ ]:
from peft import PeftModel, PeftConfig


# 2. Adapter 가중치만 포함된 디렉터리에서 PeftConfig 로드
peft_model_id = "RowKick/kanana-1.5-8b-instruct-stage2"
peft_config = PeftConfig.from_pretrained(peft_model_id)

# 3. Adapter 가중치를 base model에 로드하여 합치기
model = PeftModel.from_pretrained(model, peft_model_id)


In [ ]:
ds = Dataset.from_csv("test.csv")

In [ ]:
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        title = ["<Title>: " + t for t in batch["title"]]
        para = ["\n\n<Full text>: " + t for t in batch["paragraph_text"]]
        texts = [t + p for t, p in zip(title, para)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)

        return {**tokenized}

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=1024)
ds = ds.map(encode, batched=True)

In [ ]:
from tqdm.notebook import tqdm
from transformers.data.data_collator import pad_without_fast_tokenizer_warning


@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(ds, model, batch_size=1):
    preds = []
    pseudo = []
    model.eval()
    
    for start_idx in tqdm(range(0, len(ds), batch_size)):
        end_idx = min(start_idx + batch_size, len(ds))
        tmp = ds[start_idx:end_idx]
        input_ids = tmp["input_ids"]
        attention_mask = tmp["attention_mask"]
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to("cuda:0"))
        proba = outputs.logits.cpu()
        
        preds.extend(proba[:, 0].tolist())
        pseudo.extend(proba[:,1].tolist())
    
    return preds, pseudo

In [ ]:
a, b = inference(ds, model)

In [ ]:
import pandas as pd


sub = pd.read_csv('sample_submission.csv')
sub.head()

sub['generated'] = a

sub.to_csv('stage2_a_kanana_8b.csv', index=False)